# Sequential Channel Learning for Mobile Beam Tracking

An sequential channel learing framework for Mobile Beam Tracking Using Seq2Seq.

In [1]:
# import packages
import os
import math
import time
import json
import random
import scipy.io as sio

from collections import OrderedDict

import numpy as np
import tensorflow as tf

# from seq2seq_model import Seq2SeqModel
from seq2seq_model_new import Seq2SeqModel

In [14]:
# Loading data
Data = sio.loadmat('./data/WI_MIMO_1d_28000MHz_HO_Beam1024_TT_sp2.mat')
X_train = Data['X_train']
y_train = Data['y_train']
X_test = Data['X_test']
y_test = Data['y_test']

In [17]:
# parameters setting
train_config = {}
train_config['source_vocabulary'] = ''
train_config['target_vocabulary'] = ''
train_config['source_train_data'] = ''
train_config['target_train_data'] = ''
train_config['source_valid_data'] =  ''
train_config['target_valid_data'] =  ''

# Network parameters
train_config['input_size'] = 128
train_config['cell_type'] = 'gru'
train_config['attention_type'] =  'luong'
train_config['hidden_units'] = 256
train_config['depth'] =  2
train_config['embedding_size'] =  500
train_config['num_encoder_symbols'] = 10001
train_config['num_decoder_symbols'] = 1026

train_config['use_residual'] =  False
train_config['attn_input_feeding'] =  True
train_config['use_dropout'] =  True
train_config['dropout_rate'] =  0.5

# Training parameters
train_config['learning_rate'] = 0.0002
train_config['max_gradient_norm'] =  1.0
train_config['batch_size'] =  128
train_config['max_epochs'] = 400
train_config['max_batch'] = 10000
train_config['max_load_batches'] = 20
train_config['max_seq_length'] = 50
train_config['display_freq'] = 100
train_config['save_freq'] =  11500
train_config['valid_freq'] = 1150000
train_config['optimizer'] = 'adam'
train_config['model_dir'] =  'model_i20o20_h128_2_e500_gru_ttbs_atth_wihosp2/'
train_config['summary_dir'] = train_config['model_dir']+'/summary'
train_config['model_name'] = 'scl.ckpt'
train_config['shuffle_each_epoch'] = True
train_config['sort_by_length'] = True
train_config['use_fp16'] = False

# Runtime parameters
train_config['allow_soft_placement'] =  True
train_config['log_device_placement'] = False

# Channel Inference parameters
source_len = 20
target_len = 20
path_tic = 0
loc_tic = 0
model_load = True

In [4]:
# create model
def create_model(session, train_config):

    model = Seq2SeqModel(train_config, 'train')

    ckpt = tf.train.get_checkpoint_state(train_config['model_dir'])
    if ckpt and model_load and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print('Reloading model parameters..')
        model.restore(session, ckpt.model_checkpoint_path)
        
    else:
        if not os.path.exists(train_config['model_dir']):
            os.makedirs(train_config['model_dir'])
        print('Created new model parameters..')
        session.run(tf.global_variables_initializer())
   
    return model

# Preparation on Data
def prepare_train_batch(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len],dtype=np.int32)
    y_batch = np.zeros([batch_size,target_len],dtype=np.int32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        Temp_X.shape = -1
        Temp_y = y[0,ptic]
        Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:] = np.array(Temp_X[ltic:ltic+source_len]).reshape(-1)
                y_batch[i,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len]).reshape(-1)
                ltic = ltic + source_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:]
            y_batch = y_batch[0:flags2,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    return X_batch,X_length,y_batch,y_length,ptic,ltic

# without using embedding layers for input
def prepare_train_batch_new(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    N_BS = np.shape(X[0,0])[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len,N_BS],dtype=np.float32)
    y_batch = np.zeros([batch_size,target_len],dtype=np.int32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        # Temp_X.shape = -1
        Temp_y = y[0,ptic]
        # Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:,:] = np.array(Temp_X[ltic:ltic+source_len,:])
                y_batch[i,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len]).reshape(-1)
                ltic = ltic + target_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:]
            y_batch = y_batch[0:flags2,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    y_batch = y_batch+1
    return X_batch,X_length,y_batch,y_length,ptic,ltic

# without using embedding layers for input and output
def prepare_train_batch_neb(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    N_BS = np.shape(X[0,0])[1]
    N_beam = np.shape(y[0,0])[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len,N_BS],dtype=np.float32)
    y_batch = np.zeros([batch_size,target_len,N_beam],dtype=np.float32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        # Temp_X.shape = -1
        Temp_y = y[0,ptic]
        # Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:,:] = np.array(Temp_X[ltic:ltic+source_len,:])
                y_batch[i,:,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len,:])
                ltic = ltic + target_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:,:]
            y_batch = y_batch[0:flags2,:,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    y_batch = y_batch+1
    return X_batch,X_length,y_batch,y_length,ptic,ltic

In [18]:
# Load parallel data to train
print('Loading training data..')
if train_config['source_valid_data'] and train_config['source_valid_data']:
    print('Loading validation data..')
else:
    valid_set = None
tf.reset_default_graph()
# Initiate TF session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=train_config['allow_soft_placement'], 
    log_device_placement=train_config['log_device_placement'], gpu_options=tf.GPUOptions(allow_growth=True))) as sess:

    # Create a log writer object
    log_writer = tf.summary.FileWriter(train_config['model_dir'], graph=sess.graph)

    # Create a new model or reload existing checkpoint
    model = create_model(sess, train_config)
    step_time, loss = 0.0, 0.0
    words_seen, sents_seen = 0, 0
    start_time = time.time()

    # Training loop
    print('Training..')
    for epoch_idx in range(train_config['max_epochs']):
        if model.global_epoch_step.eval() >= train_config['max_epochs']:
            print('Training is already complete.', \
                  'current epoch:{}, max epoch:{}'.format(model.global_epoch_step.eval(), train_config['max_epochs']))
            break
        #reset data
        path_tic = 0
        loc_tic = 0
        for i_batch in range(train_config['max_batch']):    
            # Get a batch from training parallel data
            source, source_len_g, target, target_len_g,path_tic,loc_tic = prepare_train_batch_new(X_train,y_train,train_config['batch_size'],path_tic,loc_tic)
            if source is None or target is None:
                print('No samples under max_seq_length ', train_config['max_seq_length'])
                break

            # Execute a single training step
            step_loss, summary = model.train(sess, encoder_inputs=source, encoder_inputs_length=source_len_g, 
                                             decoder_inputs=target, decoder_inputs_length=target_len_g)

            loss += float(step_loss) / train_config['display_freq']
            words_seen += float(np.sum(source_len_g+target_len_g))
            sents_seen += float(source.shape[0]) # batch_size

            if model.global_step.eval() % train_config['display_freq'] == 0:

                avg_perplexity = math.exp(float(loss)) if loss < 300 else float("inf")

                time_elapsed = time.time() - start_time
                step_time = time_elapsed / train_config['display_freq']

                words_per_sec = words_seen / time_elapsed
                sents_per_sec = sents_seen / time_elapsed

                print('Epoch ', model.global_epoch_step.eval(), 'Step ', model.global_step.eval(), \
                      'Perplexity {0:.2f}'.format(avg_perplexity), 'Step-time ', step_time, \
                      '{0:.2f} pieces/s'.format(sents_per_sec), '{0:.2f} points/s'.format(words_per_sec))

                loss = 0
                words_seen = 0
                sents_seen = 0
                start_time = time.time()

                # Record training summary for the current batch
                log_writer.add_summary(summary, model.global_step.eval())

            # Execute a validation step
            if valid_set and model.global_step.eval() % train_config['valid_freq'] == 0:
                print('Validation step')
                valid_loss = 0.0
                valid_sents_seen = 0
                for source_seq, target_seq in valid_set:
                    # Get a batch from validation parallel data
                    source, source_len, target, target_len = prepare_train_batch(source_seq, target_seq)

                    # Compute validation loss: average per word cross entropy loss
                    step_loss = model.eval(sess, encoder_inputs=source, encoder_inputs_length=source_len_g,
                                           decoder_inputs=target, decoder_inputs_length=target_len_g)
                    batch_size = source.shape[0]

                    valid_loss += step_loss * batch_size
                    valid_sents_seen += batch_size
                    print(' {} samples seen'.format(valid_sents_seen))

                valid_loss = valid_loss / valid_sents_seen
                print('Valid perplexity: {0:.2f}'.format(math.exp(valid_loss)))

            # Save the model checkpoint
            '''
            if model.global_step.eval() % train_config['save_freq'] == 0:
                print('Saving the model..')
                checkpoint_path = os.path.join(train_config['model_dir'], train_config['model_name'])
                model.save(sess, checkpoint_path, global_step=model.global_step)
                json.dump(model.config,
                          open('%s-%d.json' % (checkpoint_path, model.global_step.eval()), 'w'),
                          indent=2)'''

        # Increase the epoch index of the model
        model.global_epoch_step_op.eval()
        print('Epoch {0:} DONE'.format(model.global_epoch_step.eval()))
        if(model.global_epoch_step.eval()%10==0):
            print('Saving the model..')
            checkpoint_path = os.path.join(train_config['model_dir'], train_config['model_name'])
            model.save(sess, checkpoint_path, global_step=model.global_step)
            json.dump(model.config,
                      open('%s-%d.json' % (checkpoint_path, model.global_step.eval()), 'w'),indent=2)
    '''
    print('Saving the last model..')
    checkpoint_path = os.path.join(train_config['model_dir'], train_config['model_name'])
    model.save(sess, checkpoint_path, global_step=model.global_step)
    json.dump(model.config,
              open('%s-%d.json' % (checkpoint_path, model.global_step.eval()), 'w'),
              indent=2)'''

print('Training Terminated')


Loading training data..
building model..
building encoder..
building decoder and attention..
setting optimizer..
Created new model parameters..
Training..
No samples under max_seq_length  50
Epoch 1 DONE
No samples under max_seq_length  50
Epoch 2 DONE
Epoch  2 Step  100 Perplexity 176.81 Step-time  1.3565476608276368 94.36 pieces/s 3774.29 points/s
No samples under max_seq_length  50
Epoch 3 DONE
No samples under max_seq_length  50
Epoch 4 DONE
No samples under max_seq_length  50
Epoch 5 DONE
Epoch  5 Step  200 Perplexity 18.70 Step-time  1.2151113152503967 105.34 pieces/s 4213.61 points/s
No samples under max_seq_length  50
Epoch 6 DONE
No samples under max_seq_length  50
Epoch 7 DONE
Epoch  7 Step  300 Perplexity 7.14 Step-time  1.1749197936058045 108.94 pieces/s 4357.74 points/s
No samples under max_seq_length  50
Epoch 8 DONE
No samples under max_seq_length  50
Epoch 9 DONE
No samples under max_seq_length  50
Epoch 10 DONE
Saving the model..
model saved at model_i20o20_h128_2_e500

No samples under max_seq_length  50
Epoch 83 DONE
No samples under max_seq_length  50
Epoch 84 DONE
Epoch  84 Step  3300 Perplexity 1.92 Step-time  0.8904871463775634 143.74 pieces/s 5749.66 points/s
No samples under max_seq_length  50
Epoch 85 DONE
No samples under max_seq_length  50
Epoch 86 DONE
No samples under max_seq_length  50
Epoch 87 DONE
Epoch  87 Step  3400 Perplexity 1.88 Step-time  0.9098170566558837 140.69 pieces/s 5627.50 points/s
No samples under max_seq_length  50
Epoch 88 DONE
No samples under max_seq_length  50
Epoch 89 DONE
Epoch  89 Step  3500 Perplexity 1.88 Step-time  0.9567004013061523 133.79 pieces/s 5351.73 points/s
No samples under max_seq_length  50
Epoch 90 DONE
Saving the model..
model saved at model_i20o20_h128_2_e500_gru_ttbs_atth_wihosp2/scl.ckpt-3510
No samples under max_seq_length  50
Epoch 91 DONE
No samples under max_seq_length  50
Epoch 92 DONE
Epoch  92 Step  3600 Perplexity 1.85 Step-time  0.9837264204025269 130.12 pieces/s 5204.70 points/s
No sa

No samples under max_seq_length  50
Epoch 166 DONE
Epoch  166 Step  6500 Perplexity 1.50 Step-time  0.8700447607040406 147.12 pieces/s 5884.75 points/s
No samples under max_seq_length  50
Epoch 167 DONE
No samples under max_seq_length  50
Epoch 168 DONE
No samples under max_seq_length  50
Epoch 169 DONE
Epoch  169 Step  6600 Perplexity 1.50 Step-time  0.8704254531860351 147.05 pieces/s 5882.18 points/s
No samples under max_seq_length  50
Epoch 170 DONE
Saving the model..
model saved at model_i20o20_h128_2_e500_gru_ttbs_atth_wihosp2/scl.ckpt-6630
No samples under max_seq_length  50
Epoch 171 DONE
Epoch  171 Step  6700 Perplexity 1.49 Step-time  0.9419333815574646 135.89 pieces/s 5435.63 points/s
No samples under max_seq_length  50
Epoch 172 DONE
No samples under max_seq_length  50
Epoch 173 DONE
No samples under max_seq_length  50
Epoch 174 DONE
Epoch  174 Step  6800 Perplexity 1.46 Step-time  0.8693722820281983 147.23 pieces/s 5889.31 points/s
No samples under max_seq_length  50
Epoch 

Epoch  248 Step  9700 Perplexity 1.28 Step-time  0.8707732820510864 147.00 pieces/s 5879.83 points/s
No samples under max_seq_length  50
Epoch 249 DONE
No samples under max_seq_length  50
Epoch 250 DONE
Saving the model..
model saved at model_i20o20_h128_2_e500_gru_ttbs_atth_wihosp2/scl.ckpt-9750
No samples under max_seq_length  50
Epoch 251 DONE
Epoch  251 Step  9800 Perplexity 1.28 Step-time  0.9665907430648804 132.42 pieces/s 5296.97 points/s
No samples under max_seq_length  50
Epoch 252 DONE
No samples under max_seq_length  50
Epoch 253 DONE
Epoch  253 Step  9900 Perplexity 1.27 Step-time  0.8740355706214905 146.45 pieces/s 5857.89 points/s
No samples under max_seq_length  50
Epoch 254 DONE
No samples under max_seq_length  50
Epoch 255 DONE
No samples under max_seq_length  50
Epoch 256 DONE
Epoch  256 Step  10000 Perplexity 1.27 Step-time  0.8751663637161254 146.26 pieces/s 5850.32 points/s
No samples under max_seq_length  50
Epoch 257 DONE
No samples under max_seq_length  50
Epoch

model saved at model_i20o20_h128_2_e500_gru_ttbs_atth_wihosp2/scl.ckpt-12870
Epoch  330 Step  12900 Perplexity 1.19 Step-time  0.9835967874526977 130.13 pieces/s 5205.39 points/s
No samples under max_seq_length  50
Epoch 331 DONE
No samples under max_seq_length  50
Epoch 332 DONE
No samples under max_seq_length  50
Epoch 333 DONE
Epoch  333 Step  13000 Perplexity 1.19 Step-time  0.8857738256454468 144.51 pieces/s 5780.26 points/s
No samples under max_seq_length  50
Epoch 334 DONE
No samples under max_seq_length  50
Epoch 335 DONE
Epoch  335 Step  13100 Perplexity 1.18 Step-time  0.8853935599327087 144.57 pieces/s 5782.74 points/s
No samples under max_seq_length  50
Epoch 336 DONE
No samples under max_seq_length  50
Epoch 337 DONE
No samples under max_seq_length  50
Epoch 338 DONE
Epoch  338 Step  13200 Perplexity 1.18 Step-time  0.8855936980247497 144.54 pieces/s 5781.43 points/s
No samples under max_seq_length  50
Epoch 339 DONE
No samples under max_seq_length  50
Epoch 340 DONE
Savin